In [ ]:
import xarray as xr
import hvplot.xarray
import holoviews as hv
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.ticker as mticker
from matplotlib.offsetbox import AnchoredText

from snobedo.lib.dask_utils import start_cluster, client_ip_and_port
from snobedo.snotel import SnotelLocations, CsvParser

from common import SNOBAL_DIR, DATA_DIR, SNOTEL_DIR, COARSEN_OPTS, \
    use_hvplot, HV_PLOT_OPTS, BOKEH_FONT, LEGEND_OPTS, LINE_STYLE

In [ ]:
client = start_cluster(8, 24)
client_ip_and_port(client)

## SNOTEL 

In [ ]:
year = 2022
water_year = f'wy{year}'
date_range = np.datetime64(f'{year - 1}-10-20'), np.datetime64(f'{year}-06-29')

In [ ]:
snotel_sites = SnotelLocations()
snotel_sites.load_from_json(SNOTEL_DIR / 'site-locations/snotel_sites.json')

In [ ]:
schofield_snotel_csv = pd.concat([
    CsvParser.file(
        SNOTEL_DIR / water_year / f'Schofield/usda-csv/{year}-Schofield-Pass.csv',
    )
])

butte_snotel_csv = pd.concat([
    CsvParser.file(
        SNOTEL_DIR /water_year  / f'Butte/usda-csv/{year}-Butte.csv',
    ),
])

taylor_snotel_csv = pd.concat([
    CsvParser.file(
        SNOTEL_DIR / water_year / f'Taylor/usda-csv/{year}-Upper-Taylor.csv',
    )
])

In [ ]:
irwin_csv = pd.concat([
    CsvParser.file(
        SNOTEL_DIR / water_year / f'Irwin/usda-csv/{year}-Irwin.csv',
    )
])

irwin_csv['Depth(m)'] = irwin_csv['Depth(m)'] / 10

## Model

In [ ]:
solar_HRRR = xr.open_mfdataset(
    f'{SNOBAL_DIR}/{water_year}/erw_hrrr_solar/run*/snow.nc',
    parallel=True, chunks={'time': -1},
    preprocess=lambda ds: ds['thickness']
)

butte_snobal_hrrr = solar_HRRR.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat, method='nearest').compute()
schofield_snobal_hrrr = solar_HRRR.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat, method='nearest').compute()
taylor_snobal_hrrr = solar_HRRR.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat, method='nearest').compute()
irwin_snobal_hrrr = solar_HRRR.sel(x=snotel_sites.Irwin.lon, y=snotel_sites.Irwin.lat, method='nearest').compute()

del solar_HRRR

In [ ]:
solar_HRRR = xr.open_mfdataset(
    f'{SNOBAL_DIR}/{water_year}/erw_hrrr_solar_modis/run*/snow.nc',
    parallel=True, chunks={'time': -1},
    preprocess=lambda ds: ds['thickness']
)

butte_snobal_hrrr_modis = solar_HRRR.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat, method='nearest').compute()
schofield_snobal_hrrr_modis = solar_HRRR.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat, method='nearest').compute()
taylor_snobal_hrrr_modis = solar_HRRR.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat, method='nearest').compute()
irwin_snobal_hrrr_modis = solar_HRRR.sel(x=snotel_sites.Irwin.lon, y=snotel_sites.Irwin.lat, method='nearest').compute()

del solar_HRRR

In [ ]:
solar_SMRF = xr.open_mfdataset(
    f'{SNOBAL_DIR}/{water_year}/erw/run*/snow.nc',
    preprocess=lambda ds: ds['thickness'],
    parallel=True, chunks={'time': -1},
)

butte_snobal = solar_SMRF.sel(x=snotel_sites.Butte.lon, y=snotel_sites.Butte.lat, method='nearest').compute()
schofield_snobal = solar_SMRF.sel(x=snotel_sites.Schofield.lon, y=snotel_sites.Schofield.lat, method='nearest').compute()
taylor_snobal = solar_SMRF.sel(x=snotel_sites.Taylor.lon, y=snotel_sites.Taylor.lat, method='nearest').compute()
irwin_snobal = solar_SMRF.sel(x=snotel_sites.Irwin.lon, y=snotel_sites.Irwin.lat, method='nearest').compute()

del solar_SMRF

In [ ]:
client.shutdown()

# Matplotlib 

In [ ]:
STATION_LABEL = 'Station'
COLORS = {
        'HRRR-MODIS': 'cornflowerblue',
        'HRRR': 'lightcoral',
        'Time-Decay': 'sandybrown',
        STATION_LABEL: 'seagreen',    
}

def plot_site(data, ax, site_name):
    for key in data:
        if key == STATION_LABEL:
            continue
        
        ax.plot(
            data[key].time,
            data[key].data.flatten(),
            label=key, 
            color=COLORS[key], 
            alpha=0.9, lw=1.25
        )
    ax.plot(
        data[STATION_LABEL].index,
        data[STATION_LABEL].values,
        label=key, 
        color=COLORS[STATION_LABEL], 
        alpha=0.9, lw=1.25
    ) 
    
    ax.set_ylabel(r'$\Delta$ Snow Depth (m)')

    ax.tick_params(axis='x', which='minor', tick1On=False, tick2On=False)

    ax.set_xlim([plot_range[0], plot_range[-1]])
    ax.set_yticks(np.arange(0, 4))
    ax.set_ylim(bottom=-0.1, top=3.3)
            
    at = AnchoredText(
        site_name, 
        prop=dict(size=10), 
        frameon=True, 
        loc='upper right', 
        pad=0.3, 
        borderpad=0.25,
    )
    at.patch.set_boxstyle("round", pad=0., rounding_size=0.2)
    at.patch.set(edgecolor='lightgrey')
    ax.add_artist(at)

In [ ]:
figure_opts = dict(figsize=(7,6), dpi=300,)
plot_range = pd.date_range(start=f'{year - 1}-10-01', periods=11, freq='MS')
xTicks = mdates.DateFormatter('%b')

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True, **figure_opts)
fig.set_facecolor('whitesmoke')
plt.subplots_adjust(hspace=0.05)

plot_site(
    {
        'HRRR-MODIS': butte_snobal_hrrr_modis['thickness'],
        'HRRR': butte_snobal_hrrr['thickness'],
        'Time-Decay': butte_snobal['thickness'],
        STATION_LABEL: butte_snotel_csv['Depth(m)'],
    },
    axes[0],
    'Butte'
)
aso_marker=dict(label='ASO', marker='o', c='black', s=10)
axes[0].scatter(
    [np.datetime64('2022-04-21'), np.datetime64('2022-05-18')],
    [0.53, 0.03],
    **aso_marker
)

plot_site(
    {
        'HRRR-MODIS': schofield_snobal_hrrr_modis['thickness'],
        'HRRR': schofield_snobal_hrrr['thickness'],
        'Time-Decay': schofield_snobal['thickness'],
        STATION_LABEL: schofield_snotel_csv['Depth(m)'],
    },
    axes[1],
    'Schofield Pass'
)
axes[1].scatter( [], [], **aso_marker)

axes[1].xaxis.set_major_locator(mdates.MonthLocator())
axes[1].xaxis.set_minor_locator(mdates.MonthLocator(bymonthday=16))
axes[1].xaxis.set_major_formatter(mticker.NullFormatter())
axes[1].xaxis.set_minor_formatter(xTicks)
axes[1].tick_params(axis='x', which='minor', pad=-0.5)

at = AnchoredText(
    f'Water Year {year}', 
    prop=dict(size=10), 
    frameon=False, 
    loc='upper left', 
    pad=0.3, 
    borderpad=0.25,
)
axes[0].add_artist(at)
axes[1].legend(
    frameon=False,
    bbox_to_anchor=(0.95, -0.125),
    ncol=5,
    borderaxespad=0.15, 
    fontsize=8
);

# HV plot

## SNOTEL 

In [ ]:
use_hvplot()

In [ ]:
hv.output(fig='auto', dpi=300)

HV_PLOT_OPTS['ylabel'] = 'Snow Depth (m)'
HV_PLOT_OPTS['xlabel'] = f'Water Year {year}'

y_lim = (-0.1, 3.5)
SNOTEL_LINE=dict(line_width=3)

In [ ]:
butte_snobal_hrrr_modis.thickness.squeeze(['x', 'y']).hvplot(label='HRRR + MODIS').opts(
    title='Butte', xlim=date_range, ylim=y_lim, **LINE_STYLE, **HV_PLOT_OPTS, **BOKEH_FONT,
) * \
butte_snobal_hrrr.thickness.squeeze(['x', 'y']).hvplot(label='HRRR DSWRF', **LINE_STYLE) * \
butte_snobal.thickness.squeeze(['x', 'y']).hvplot(label='Time-Decay', **LINE_STYLE) *\
butte_snotel_csv['Depth(m)'].plot(label='Station', **SNOTEL_LINE) * \
hv.Scatter([(np.datetime64('2022-04-21'), 0.53)], label='ASO Depth').opts(color='k', size=10) * \
hv.Scatter([(np.datetime64('2022-05-18'), 0.03)], label='ASO Depth').opts(color='k', size=10)

In [ ]:
(schofield_snobal_hrrr_modis.thickness.squeeze(['x', 'y']).hvplot(label='HRRR + MODIS').opts(
    title='Schofield Pass', ylim=y_lim, **LINE_STYLE, **HV_PLOT_OPTS, **BOKEH_FONT
) * \
schofield_snobal_hrrr.thickness.squeeze(['x', 'y']).hvplot(label='HRRR DSWRF', **LINE_STYLE) * \
schofield_snobal.thickness.squeeze(['x', 'y']).hvplot(label='SMRF Time-Decay Albedo', **LINE_STYLE) * \
schofield_snotel_csv['Depth(m)'].plot(label='Station', **SNOTEL_LINE)).opts(show_legend=False,)

### Upper Taylor 

In [ ]:
taylor_snobal_hrrr_modis.thickness.squeeze(['x', 'y']).hvplot(label='HRRR + MODIS (v03)').opts(
    title='Upper Taylor', xlim=date_range, ylim=y_lim, **LINE_STYLE, **HV_PLOT_OPTS
) * \
taylor_snobal_hrrr.thickness.squeeze(['x', 'y']).hvplot(label='HRRR DSWF', **LINE_STYLE) * \
taylor_snobal.thickness.squeeze(['x', 'y']).hvplot(label='SMRF Time-Decay Albedo', **LINE_STYLE) * \
taylor_snotel_csv['Depth(m)'].plot(label='Station', **SNOTEL_LINE)

### Irwin 

***NOTE***: Irwin guide is not a SNOTEL station and does not have a QA/QC end of day value. Using a 'rolling daily' window to plot

In [ ]:
irwin_snobal_hrrr_modis.thickness.squeeze(['x', 'y']).hvplot(label='HRRR + MODIS (v03)').opts(
    title='Irwin Guide', xlim=date_range, ylim=y_lim, **LINE_STYLE, **HV_PLOT_OPTS
) * \
irwin_snobal_hrrr.thickness.squeeze(['x', 'y']).hvplot(label='HRRR DSWRF', **LINE_STYLE) * \
irwin_snobal.thickness.squeeze(['x', 'y']).hvplot(label='SMRF Time-Decay Albedo', **LINE_STYLE) * \
irwin_csv['Depth(m)'].rolling('1D').mean().hvplot(label='Station', **SNOTEL_LINE)